# **Requirements**

In [44]:
!pip install tensorflow-gpu

In [45]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# **Transfer Learning**


 

---



Importing the libraries

In [46]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.inception_v3  import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Image Resize

In [48]:
IMAGE_SIZE = [224, 224]

Import the dataset

In [49]:
train_path = '/content/drive/MyDrive/Covid19/train'
test_path = '/content/drive/MyDrive/Covid19/test'

Import the Vgg 16 library and adding preprocessing layer to the front of VGG 

In [50]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [51]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [52]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Covid19/train/*')
folders

['/content/drive/MyDrive/Covid19/train/affected',
 '/content/drive/MyDrive/Covid19/train/not affected']

In [53]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

Model creation

In [54]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [55]:
# view the structure of the model
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

Choose the best parameters

In [56]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Use the Image Data Generator to import the images from the dataset

In [57]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [58]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Covid19/train',
                                                 target_size = (224,224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 2400 images belonging to 2 classes.


In [59]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Covid19/test',
                                            target_size = (224,224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 600 images belonging to 2 classes.


**fit the model**

In [60]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
150/150 [==============================] - 354s 2s/step - loss: 0.7618 - accuracy: 0.9521 - val_loss: 0.2004 - val_accuracy: 0.9683
Epoch 2/20
150/150 [==============================] - 350s 2s/step - loss: 0.2559 - accuracy: 0.9775 - val_loss: 0.1326 - val_accuracy: 0.9900
Epoch 3/20
150/150 [==============================] - 350s 2s/step - loss: 0.2599 - accuracy: 0.9833 - val_loss: 0.1519 - val_accuracy: 0.9850
Epoch 4/20
150/150 [==============================] - 350s 2s/step - loss: 0.2181 - accuracy: 0.9846 - val_loss: 0.5104 - val_accuracy: 0.9683
Epoch 5/20
150/150 [==============================] - 348s 2s/step - loss: 0.2419 - accuracy: 0.9837 - val_loss: 0.2624 - val_accuracy: 0.9783
Epoch 6/20
150/150 [==============================] - 350s 2s/step - loss: 0.3374 - accuracy: 0.9842 - val_loss: 0.6880 - val_accuracy: 0.9717
Epoch 7/20
150/150 [==============================] - 349s 2s/step - loss: 0.3494 - accuracy: 0.9829 - val_loss: 0.2638 - val_accuracy: 0.9900

**Save it as a h5 file**

In [61]:
from tensorflow.keras.models import load_model

model.save('model_vgg_Covid19.h5')

Apply model on test set

In [62]:
y_pred = model.predict(test_set)

In [63]:
y_pred

array([[9.9999285e-01, 7.1986015e-06],
       [9.8802465e-01, 1.1975387e-02],
       [1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [64]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [65]:
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,

In [67]:
model=load_model('model_vgg_Covid19.h5')

In [97]:
img=image.load_img('/content/drive/MyDrive/Covid19/test/affected/COVID_1202.png',target_size=(224,224))

In [98]:
x=image.img_to_array(img)
x

array([[[185., 185., 185.],
        [172., 172., 172.],
        [168., 168., 168.],
        ...,
        [176., 176., 176.],
        [178., 178., 178.],
        [164., 164., 164.]],

       [[ 79.,  79.,  79.],
        [ 59.,  59.,  59.],
        [ 56.,  56.,  56.],
        ...,
        [ 60.,  60.,  60.],
        [ 62.,  62.,  62.],
        [ 52.,  52.,  52.]],

       [[ 33.,  33.,  33.],
        [ 12.,  12.,  12.],
        [  5.,   5.,   5.],
        ...,
        [  5.,   5.,   5.],
        [  7.,   7.,   7.],
        [  3.,   3.,   3.]],

       ...,

       [[155., 155., 155.],
        [195., 195., 195.],
        [208., 208., 208.],
        ...,
        [133., 133., 133.],
        [142., 142., 142.],
        [155., 155., 155.]],

       [[169., 169., 169.],
        [193., 193., 193.],
        [205., 205., 205.],
        ...,
        [150., 150., 150.],
        [154., 154., 154.],
        [157., 157., 157.]],

       [[216., 216., 216.],
        [215., 215., 215.],
        [212., 2

In [99]:
x.shape

(224, 224, 3)

In [100]:
x=x/255

In [101]:
import numpy as np
x=np.expand_dims(x,axis=0)
print(x)
img_data=preprocess_input(x)
img_data.shape

[[[[0.7254902  0.7254902  0.7254902 ]
   [0.6745098  0.6745098  0.6745098 ]
   [0.65882355 0.65882355 0.65882355]
   ...
   [0.6901961  0.6901961  0.6901961 ]
   [0.69803923 0.69803923 0.69803923]
   [0.6431373  0.6431373  0.6431373 ]]

  [[0.30980393 0.30980393 0.30980393]
   [0.23137255 0.23137255 0.23137255]
   [0.21960784 0.21960784 0.21960784]
   ...
   [0.23529412 0.23529412 0.23529412]
   [0.24313726 0.24313726 0.24313726]
   [0.20392157 0.20392157 0.20392157]]

  [[0.12941177 0.12941177 0.12941177]
   [0.04705882 0.04705882 0.04705882]
   [0.01960784 0.01960784 0.01960784]
   ...
   [0.01960784 0.01960784 0.01960784]
   [0.02745098 0.02745098 0.02745098]
   [0.01176471 0.01176471 0.01176471]]

  ...

  [[0.60784316 0.60784316 0.60784316]
   [0.7647059  0.7647059  0.7647059 ]
   [0.8156863  0.8156863  0.8156863 ]
   ...
   [0.52156866 0.52156866 0.52156866]
   [0.5568628  0.5568628  0.5568628 ]
   [0.60784316 0.60784316 0.60784316]]

  [[0.6627451  0.6627451  0.6627451 ]
   [0.7

(1, 224, 224, 3)

In [102]:
model.predict(img_data)

array([[1.00000e+00, 6.75889e-14]], dtype=float32)

In [103]:
a=np.argmax(model.predict(img_data), axis=1)
a

array([0])

In [95]:
if(a==0):
    print("affected")
else:
    print("Uninfected")

affected
